In [1]:
import polars

df = polars.read_csv("./data/Trade_DetailedTradeMatrix.csv", encoding='latin-1')

In [16]:
colnames = []
for i in df.columns:
    i = i.replace(" ", "_")
    colnames.append(i)
    print(i)

df.columns = colnames

Reporter_Country_Code
Reporter_Country_Code_(M49)
Reporter_Countries
Partner_Country_Code
Partner_Country_Code_(M49)
Partner_Countries
Item_Code
Item_Code_(CPC)
Item
Element_Code
Element
Year_Code
Year
Unit
Value
Flag


This is the part where we analyze the data.

In [46]:
import duckdb

duckdb.sql('''
    select
        round(sum(Value), 2) as tot
        from df
        where Year = 2019
        and Reporter_Countries = 'United Arab Emirates' and
        Element = 'Import Quantity';
''')

┌─────────────┐
│     tot     │
│   double    │
├─────────────┤
│ 17213036.22 │
└─────────────┘

In [57]:
import duckdb
import polars

duckdb.sql('''
    select Partner_Countries,
        sum(Value) as trade_value,
        round(trade_value /
        (
            select
                sum(Value)
            from df
            where Year = 2019
            and Reporter_Countries = 'United Arab Emirates' and
            Element = 'Import Quantity'
        ), 2) as dep_ratio
    from df
    where Year = 2019
    and Reporter_Countries = 'United Arab Emirates'
    and Element = 'Import Value'
    and Value > 0
    group by Partner_Countries
    having dep_ratio >= 0.05
    order by trade_value desc;
''').pl()